# <font color='blue'>Valis slide-alignment for SignalStar</font>

## Imports

In [2]:
from pathlib import Path
import time
import os
import numpy as np
from valis import registration
from valis.micro_rigid_registrar import MicroRigidRegistrar # For high resolution rigid registration

from tifffile import TiffFile
from ome_types import from_xml
from valis import valtils

C:\Anaconda3\envs\valis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Settings

<div class="alert alert-block alert-info">
    <b>ometiffs_paths</b> should contain folders for set of images that will be merged, ideally created using the previous notebook <i>VSI_cropping.ipynb</i>
</div>

In [25]:
ometiffs_paths = 'ometiffs'
ometiffs_paths = Path(ometiffs_paths)

# Whether to use micro or macro registration. Look at the Valis documentaiton for details: https://valis.readthedocs.io/en/latest/examples.html#high-resolution-registration
registration_method = 'micro' # or 'macro'

# Fraction full resolution used for non-rigid registration in micro-mode
micro_reg_fraction = 0.25 

## Run registration and merging

In [ ]:
for f in ometiffs_paths.iterdir():
    
    # ------------
    # Set up parameters
    # ------------
    
    slide_name = f.stem
    slide_src_dir = f
    results_dst_dir = "./registered_slides"
    registered_slide_dst_dir = f"./registered_slides/{slide_name}"
    merged_slide_dst_f = f"./merged/{slide_name}.ome.tiff"

    # ------------
    # Registration
    # ------------
    
    # Normal (non-micro) registration
    if registration_method=='macro':
        
        # Create a Valis object and use it to register the slides in slide_src_dir
        registrar = registration.Valis(slide_src_dir, results_dst_dir)
        rigid_registrar, non_rigid_registrar, error_df = registrar.register()
    

    # Micro registration        
    elif registration_method=='micro':
        
        # Perform high resolution rigid registration using the MicroRigidRegistrar
        start = time.time()
        registrar = registration.Valis(slide_src_dir, results_dst_dir, micro_rigid_registrar_cls=MicroRigidRegistrar)
        rigid_registrar, non_rigid_registrar, error_df = registrar.register()

        # Calculate what `max_non_rigid_registration_dim_px` needs to be to do non-rigid registration on an image that is 25% full resolution.
        img_dims = np.array([slide_obj.slide_dimensions_wh[0] for slide_obj in registrar.slide_dict.values()])
        min_max_size = np.min([np.max(d) for d in img_dims])
        img_areas = [np.multiply(*d) for d in img_dims]
        max_img_w, max_img_h = tuple(img_dims[np.argmax(img_areas)])
        micro_reg_size = np.floor(min_max_size*micro_reg_fraction).astype(int)

        # Perform high resolution non-rigid registration using 20% full resolution
        micro_reg, micro_error = registrar.register_micro(max_non_rigid_registration_dim_px=micro_reg_size)
        
    # --------------
    # Channel names
    # --------------
    
    def get_channel_names_ome_tiff(path):
        # Helper function to get channel names from tiff files
        with TiffFile(path) as tif:
            ome_xml = tif.ome_metadata
            ome = from_xml(ome_xml)
            return [channel.name for channel in ome.images[0].pixels.channels]

    def cnames_from_filename(src_f):
        # Clean up channel names specifically as they come out from Olympus VS200
        channels = get_channel_names_ome_tiff(src_f)
        channels = [c.replace('KSS ','') for c in channels]

        f = valtils.get_name(src_f)
        f = f.split("_")[-1]
        return [f+" "+c for c in channels]

    channel_name_dict = {f:cnames_from_filename(f) for f in registrar.original_img_list}
    
    # ------------
    # Registration
    # ------------
    
    merged_img, channel_names, ome_xml = \
        registrar.warp_and_merge_slides(merged_slide_dst_f,
                                        channel_name_dict=channel_name_dict,
                                        drop_duplicates=True)

registration.kill_jvm() # Kill the JVM